In [1]:
import numpy as np
from astropy.table import Table, join, MaskedColumn, vstack
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy
from astropy.time import Time
import pandas as pd
import re
import seaborn as sns
import datetime
from datetime import datetime
from datetime import timedelta
from math import e
from math import pi
from astropy.table import Column
from math import sqrt
import numpy as np
import emcee
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
import math
import corner
from numpy import exp
from scipy import integrate
from scipy.integrate import quad
import pdb
import powerlaw
import random
from scipy import stats
from scipy.optimize import curve_fit
import scipy.optimize as opt
from multiprocessing import Pool

In [2]:
Cat=Table.read('M33_catalog_with_Quick_BestFit_CMD_Estimates.fits')

subimg_data=Table.read('phatm33_subimg-meta.fits')
subimg_data.rename_column('SUBIMG', 'IMG')
subimg_data.rename_column('FILENAME', 'SUBIMG_old')
subimg_data.rename_column('RA', 'IMG_RA')
subimg_data.rename_column('DEC', 'IMG_DEC')

si_lst=list(subimg_data['SUBIMG_old'])
for i in range(len(si_lst)):
    si_lst[i]= si_lst[i].rstrip()
    
new_col=Column(si_lst, name='SUBIMG')
subimg_data.remove_column('SUBIMG_old')
subimg_data.add_column(new_col, index=4)

Cat_w_nms=join(Cat, subimg_data)

Cat_w_Est=Cat_w_nms[np.where(Cat_w_nms['Quick_BestFit_LogAge'] > 0)]


full_masses=np.array(Cat_w_Est['Quick_BestFit_LogMass'])
full_ages=np.array(Cat_w_Est['Quick_BestFit_LogAge'])
full_nmses=np.array(np.log10(Cat_w_Est['NMS']))

Draw a Scechter Function with a -2 powerlaw

In [3]:
def Cliff_pl_draw_schechter(ndraw,alpha,llim,ulim,M_c=1.e4,rseed=None,returnfull=False):
    alphapp=alpha+1.
    np.random.seed(rseed)
    rand=np.random.rand(ndraw)
    pdf=(rand*ulim**alphapp + (1.-rand)*llim**alphapp)**(1./alphapp)
    rand=np.random.rand(ndraw)
    select = rand < np.exp(-pdf/M_c)
    if returnfull:
        return pdf[select], pdf
    else:
        return pdf[select]
    
drawn_masses=np.log10(Cliff_pl_draw_schechter(4000, -2, 300, 10**7, M_c=10**4.25 ))

#Draw_ages and NMS with replacement from our distributions

drawn_ages=np.random.choice(full_ages, size=len(drawn_masses))
drawn_nms=np.random.choice(full_nmses, size=len(drawn_masses))

Apply a completeness to determine detection

1. Asses the completeness for a given cluster, given its mass, age, and NMS
2. Draw a random number between 0:1 and if the number is less that the completeness value, the cluster was "Detected"


In [6]:
#Defining the Compleness funcitons

def pobs(M, mlim):
    k=6.3665
    
    y=(1.+ exp(-k*(M-mlim)))**(-1)
    
    return y

def c(NMS):
    m=0.7117385589429568
    b=0.6066972150830925
    
    y= (m*NMS)+b
    
    if NMS < 2.53:
        return 2.413
    if 2.53 <= NMS <= 3.49: 
        return y
    if NMS > 3.49:
        return 3.054

def M_lim(Tau, NMS):
    
    #fit from completeness limit 
    a=0.06005753215407492
    b=1.0190688706002926
    
    c_=c(NMS)
    
    Tau_min=6.71

    y= a*np.exp(b*(Tau-Tau_min))+c_
    return y

In [7]:
drawn_mlims=np.zeros((len(drawn_masses)))
for i in range(len(drawn_masses)):
    drawn_mlims[i]=M_lim(drawn_ages[i], drawn_nms[i])


#determin "detected" clusters
detected_masses=[]
detected_mlims=[]
for i in range(len(drawn_masses)):
    rand= np.random.rand()
    if rand < pobs(drawn_masses[i], drawn_mlims[i]):
        detected_masses.append(drawn_masses[i])
        detected_mlims.append(drawn_mlims[i])

detected_masses=np.array(detected_masses)
detected_mlims=np.array(detected_mlims)


In [8]:
# only feed in detected clusters, and only take above the 50% compleness and re-run

using_dm=detected_masses[np.where(detected_masses > detected_mlims)]
using_mlims=10**detected_mlims[np.where(detected_masses > detected_mlims)]

In [9]:
#Definging necesary funcitons 

def lnobs_like(M, mlim):
    k=6.3665
    
    return -np.log(1.+ exp(-k*(M-mlim)))
        
def Shecter_Z(M, mlim, alpha, M_c):
    x = M/M_c
    k=6.3665
    pobs= 1./(1.+ exp((-k)*(np.log10(M)-mlim)))
    
    return (x**alpha) * exp(-x) * pobs
        
    
def lnlike(theta, M, mlim):
    alpha, M_c = theta
    lin_M_c= 10.**M_c
    
    lin_M= 10**M
    x= lin_M/lin_M_c
    
    
    ln_pobs=lnobs_like(M, np.log10(mlim))
    
    norm= np.zeros(len(M))
    err=np.zeros(len(M))
    for i in range(len(M)):
        
        norm[i], err[i] = quad(Shecter_Z, mlim[i], 1.e7, args=(np.log10(mlim[i]), alpha, lin_M_c))
    
    
    lnlike = np.sum((-x) + alpha*np.log(x) + ln_pobs - np.log(norm))    
    return lnlike
    
def lnprior(theta):
    alpha, M_c = theta
    if -3 <= alpha <= -1 and 3 <= M_c <= 8:
        return 0.0
    return -np.inf

def lnprob(theta, M, mlim):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, M, mlim)

In [10]:
#Running the Maximum liklihood Fit

nll = lambda *args: -lnprob(*args)
starting_point=np.array([-2., 4.25])

fd_result=opt.minimize(nll, x0=starting_point, args=(using_dm, using_mlims))
fd_result['x']

array([-1.91087179,  4.17491414])